### Handle missing exercisebodypart values

I found the distinct equipment names for which we dont have the body part that they correspond to and with ***ChatGPT*** I manually assigned it to categories:
- eg Leg Extension Sel to LowerBody

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
import os

dataframes = {}
data_folder = 'data_cleaned/'
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        dataframes[filename[:-4]] = df
exercises = dataframes['processed_exercises']
exercises.sample(3)

/var/folders/3f/hpgbw8dd459305qvgctxmywc0000gn/T/ipykernel_80636/2983689177.py:15: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/3f/hpgbw8dd459305qvgctxmywc0000gn/T/ipykernel_80636/2983689177.py:15: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,CloudId,Gender,Age,PhysicalActivityMacroTypeName,ExerciseName,EquipmentName,DoneOnUTC,Duration_sec,Calories,MetsMin,ExerciseBodyPartName,ExerciseMusclesName,METs,EquipmentCategory
1432518,691b06333b80ddae5c50aca4969a2180bbab9479,M,27,Isotonic,Back extension with weight,Lower Back Bench,2022-07-06 18:51:34.417000+00:00,450,19.0,NaN,Core,LowerBack,1.6,Strength Training
477216,05db5b0d2f88dcf7b2f8e30a0d5fb2924984115a,M,71,Isotonic,Pulldown - close grip,Vertical Traction Sel,2022-02-21 07:49:12.967000+00:00,220,32.0,6.1,NaN,NaN,NaN,Strength Training
2720780,7c6ebe08d509f22bea78a8e8e7c6443fb601245c,M,28,Isotonic,Pull,Low Row Sel,2022-04-25 10:15:04.128000+00:00,172,18.0,NaN,UpperBody,LateralsBack,5.1,Strength Training


In [5]:
exercises.isnull().sum()

CloudId                                0
Gender                                 0
Age                                    0
PhysicalActivityMacroTypeName          0
ExerciseName                           0
EquipmentName                          3
DoneOnUTC                              0
Duration_sec                           0
Calories                               3
MetsMin                          2456592
ExerciseBodyPartName              783304
ExerciseMusclesName               863014
METs                              870238
EquipmentCategory                      3
dtype: int64

In [3]:
distinct_equipment_names = exercises[exercises['ExerciseBodyPartName'].isnull()]['EquipmentName'].unique()
print(distinct_equipment_names)

['Leg Curl Biostr' 'Leg Press Biostr' 'Leg Extension Sel'
 'Cables Crossed Elt+' 'Dumbbells' 'Barbell' 'Body weight' 'Kettlebell'
 'Wellness Ball' 'Climb Excite 1000' 'Skillrow' 'Skillrun'
 'Chest Press Sel' 'Vertical Tr. Artis' 'Total Abd. Artis'
 'Abductor Artis' 'Synchro Excite 1000' 'Climb Excite Live' 'Tubes'
 'Cable Stations 4' 'Row Pure' 'Chest Press Pure' 'Run Excite 1000'
 'Synchro Excite 700' 'Arm Extension Sel' 'Synchro Artis'
 'Bike Excite 1000' 'Run Excite Live' 'Rear Kick Pure' 'Leg Press Sel'
 'Total Abdominal Sel' 'Lower Back Sel' 'Synchro Artis 2019' 'Multipower'
 'Pectoral Sel' 'Shoulder Press Sel' 'Anterior Flexability'
 'Run Excite 600' 'Pulley Sel' 'Cables Crossed Sel' 'Total Abdom. Biostr'
 'Vertical Tr. Biostr' 'Crossover Cables' 'Recline' 'Wave'
 'Lat Machine Sel' 'Arm Extension Artis' 'Shoulder Press Artis'
 'Low Row Artis' 'Cable Jungle' 'Lower Back Bench' 'Leg Curl Elt+'
 'Leg Curl Sel' 'Adductor Artis' 'Chest Press Biostr' 'Low Row Biostr'
 'Step/Squat Kst' 

In [ ]:
# Convert the numpy array to a pandas DataFrame
distinct_equipment_df = pd.DataFrame(distinct_equipment_names, columns=['EquipmentName'])
distinct_equipment_df.to_csv('data_cleaned/distinctequipmentnamesfornullbodypart.csv', index=False)

In [16]:
exercises['ExerciseBodyPartName'].value_counts()

ExerciseBodyPartName
LowerBody    889990
UpperBody    866923
TotalBody    409057
Core         366666
Name: count, dtype: int64

Used chat to make the categorization

In [22]:
import re
# Define the path to the file
file_path = 'data_cleaned/body_part_categorized.csv'

# Initialize an empty dictionary to store the equipment-category map
bp_categories = {}

# Open and read the file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Process each line
for line in lines:
    line = line.strip()  # Remove extra spaces or newline characters
    if line:
        print(line)  # Check if the line is not empty
        equipment, bpcategory = line.rsplit(':', 1)
        bp_categories[equipment.strip()] = bpcategory.strip()

# Print or return the resulting map
print(bp_categories)

Leg Curl Biostr:LowerBody
Leg Press Biostr:LowerBody
Leg Extension Sel:LowerBody
Cables Crossed Elt+:UpperBody
Dumbbells:UpperBody
Barbell:UpperBody
Body weight:TotalBody
Kettlebell:TotalBody
Wellness Ball:Core
Climb Excite 1000:LowerBody
Skillrow:TotalBody
Skillrun:LowerBody
Chest Press Sel:UpperBody
Vertical Tr. Artis:UpperBody
Total Abd. Artis:Core
Abductor Artis:LowerBody
Synchro Excite 1000:LowerBody
Climb Excite Live:LowerBody
Tubes:UpperBody
Cable Stations 4:UpperBody
Row Pure:UpperBody
Chest Press Pure:UpperBody
Run Excite 1000:LowerBody
Synchro Excite 700:LowerBody
Arm Extension Sel:UpperBody
Synchro Artis:LowerBody
Bike Excite 1000:LowerBody
Run Excite Live:LowerBody
Rear Kick Pure:LowerBody
Leg Press Sel:LowerBody
Total Abdominal Sel:Core
Lower Back Sel:LowerBody
Synchro Artis 2019:LowerBody
Multipower:UpperBody
Pectoral Sel:UpperBody
Shoulder Press Sel:UpperBody
Anterior Flexability:Core
Run Excite 600:LowerBody
Pulley Sel:UpperBody
Cables Crossed Sel:UpperBody
Total Abdom.

In [23]:
# Filter rows where ExerciseBodyPartName is null
null_bodypart_df = exercises[exercises['ExerciseBodyPartName'].isnull()]

# Check if EquipmentName is not in bp_categories keys
not_in_bp_categories = null_bodypart_df[~null_bodypart_df['EquipmentName'].isin(bp_categories.keys())]

# Get the count of such rows
count_not_in_bp_categories = not_in_bp_categories.shape[0]
print(count_not_in_bp_categories)
print(not_in_bp_categories)

0
Empty DataFrame
Columns: [CloudId, Gender, Age, PhysicalActivityMacroTypeName, ExerciseName, EquipmentName, DoneOnUTC, Duration_sec, Calories, MetsMin, ExerciseBodyPartName, ExerciseMusclesName, METs, EquipmentCategory]
Index: []


In [24]:
# Fill the ExerciseBodyPartName column based on the bp_categories dictionary
exercises['ExerciseBodyPartName'] = exercises.apply(
    lambda row: row['ExerciseBodyPartName'] if pd.notnull(row['ExerciseBodyPartName']) else bp_categories.get(row['EquipmentName'], None),
    axis=1
)

In [25]:
exercises.isnull().sum()

CloudId                                0
Gender                                 0
Age                                    0
PhysicalActivityMacroTypeName          0
ExerciseName                           0
EquipmentName                          3
DoneOnUTC                              0
Duration_sec                           0
Calories                               3
MetsMin                          2456592
ExerciseBodyPartName                   0
ExerciseMusclesName               863014
METs                              870238
EquipmentCategory                      3
dtype: int64

In [ ]:
exercises.to_csv('data_cleaned/processed_exercises_v2.csv', index=False)